#Assignment #6

Vision Transformer

---


**SUBMISSION INSTRUCTIONS**

First make a copy of this colab file and then solve the assignment and upload your final notebook on github.

Before uploading your downloaded notebook, RENAME the file as rollno_name.ipynb

Submission Deadline : 31/01/2026 Saturday EOD i.e before 11:59 PM

The deadline is strict, Late submissions will incur penalty

Note that you have to upload your solution on the github page of the project Vision Transformer and **under Assignment 6**

And remember to keep title of your pull request to be ViT_name_rollno_assgn6

Github Submission repo -
https://github.com/electricalengineersiitk/Winter-projects-25-26/tree/main/Vision%20transformer/Assignment%206

# Part A - Data and tokens

Q1. Build a tiny toy dataset with pandas
Create a pandas DataFrame with columns text and label.
- Include at least 12 short sentences (3-10 words each).
- The label is 0/1 (e.g., positive vs negative sentiment).
- Shuffle rows and split into train/test (80/20) using a fixed random seed.
Return: df_train, df_test.

In [4]:
import pandas as pd
def make_toy_dataset(seed: int = 42):
 """Return df_train, df_test with columns: text (str), label (int)."""
 # TODO
 np.random.seed(seed)
 data = [
        ("The movie was absolutely wonderful and thrilling.", 1),
        ("I hated every moment of this terrible film.", 0),
        ("What a fantastic performance by the lead actor!", 1),
        ("The plot was boring and completely predictable.", 0),
        ("I will never watch this garbage again.", 0),
        ("Truly a masterpiece of modern cinema.", 1),
        ("The pacing was slow and the dialogue was bad.", 0),
        ("A delightful experience from start to finish.", 1),
        ("Waste of time and money, do not watch.", 0),
        ("Highly recommended for all ages, great fun.", 1),
        ("The script made no sense whatsoever.", 0),
        ("An excellent story with great visuals.", 1),
        ("Just awful, I walked out halfway through.", 0),
        ("Brilliant direction and amazing sound design.", 1)
    ]
 df = pd.DataFrame(data,colums=["text","label"])
 df = df.sample(frac=1,random_state=seed).reset_index(drop=True)
 split_idx = int(len(df) * 0.8)
 df_train = df.iloc[:split_idx].reset_index(drop=True)
 df_test = df.iloc[split_idx:].reset_index(drop=True)
 return df_train, df_test

Q2. Clean and tokenize text

Implement a basic cleaner: lowercase, strip, replace multiple spaces with one, and remove punctuation
(.,!?;:).
Tokenize by whitespace.
Add a new column tokens that stores a list of tokens per row.
Return the updated DataFrame.

In [5]:
import re
import pandas as pd
def clean_text(s: str) -> str:
 s = s.lower()
 s = re.sub(r'[.,!?:;]', '', s)
 s = re.sub(r'\s+', ' ', s).strip()
 return s
def add_tokens_column(df: pd.DataFrame) -> pd.DataFrame:
 """Adds df['tokens'] = list[str]."""
 df['tokens'] = df['text'].apply(lambda x: clean_text(x).split())
 return df


Q3. Build a vocabulary + token/id mappings

Build token2id and id2token using the training tokens.
Include special tokens: [PAD], [UNK], [BOS], [EOS] at the beginning.
Add tokens that occur at least min_freq times.
Return: token2id (dict), id2token (list).

In [6]:
from collections import Counter
from typing import Dict, List
SPECIALS = ['[PAD]', '[UNK]', '[BOS]', '[EOS]']
def build_vocab(list_of_token_lists, min_freq: int = 1):
  SPECIALS = ['[PAD]', '[UNK]', '[BOS]', '[EOS]']

  counter = Counter()
  for tokens in list_of_token_lists:
    counter.update(tokens)

  valid_tokens = [t for t, c in counter.items() if c >= min_freq]

  id2token = SPECIALS + valid_tokens
  token2id = {t: i for i, t in enumerate(id2token)}

  return token2id, id2token


Q4. Convert tokens to ids + pad to a batch

Implement tokens_to_ids for one sequence.
Implement pad_batch that takes a list of id sequences and returns:
- X: int array (B,T) padded with pad_id
- pad_mask: bool array (B,T) where True means 'real token' and False means padding

In [7]:
import numpy as np
def tokens_to_ids(tokens, token2id, unk_token='[UNK]'):
 unk_id = token2id.get(unk_token)
 return [token2id.get(t, unk_id) for t in tokens]
def pad_batch(id_seqs, pad_id: int):
 """Return X (B,T) and pad_mask (B,T)"""
 max_len = max(len(seq) for seq in id_seqs)

 B = len(id_seqs)
 T = max_len

 X = np.full((B, T), pad_id, dtype=int)

 pad_mask = np.zeros((B, T), dtype=bool)

 for i, seq in enumerate(id_seqs):
    length = len(seq)
    X[i, :length] = seq
    pad_mask[i, :length] = True
 return X, pad_mask


#Part B - Core Transformer math

Q5. Embedding lookup

Implement an embedding table E of shape (V,D) initialized from a normal distribution (mean 0, std 0.02).
Given token ids X (B,T), return embeddings of shape (B,T,D) using NumPy indexing.


In [8]:
import numpy as np
def init_embeddings(vocab_size: int, d_model: int, seed: int = 0):
 np.random.seed(seed)
 E = np.random.normal(0, 0.02, (vocab_size, d_model))
 return E
def embed(X: np.ndarray, E: np.ndarray):
 """X: (B,T) int, E: (V,D) -> out: (B,T,D)."""
 # TODO
 out = E[X]
 return out


Q6. Sinusoidal positional encoding

Implement the classic sinusoidal positional encoding PE of shape (T,D).
Then add it to token embeddings (B,T,D).
Make sure your implementation works for both even and odd D.

In [10]:
import numpy as np
def sinusoidal_positional_encoding(T: int, D: int):
 """Return PE: (T,D)."""
 PE = np.zeros((T, D))
 position = np.arange(T)[:, np.newaxis]
 div_term = np.exp(np.arange(0, D, 2) * -(np.log(10000.0) / D))

 PE[:, 0::2] = np.sin(position * div_term)

 PE[:, 1::2] = np.cos(position * div_term)
 return PE
def add_positional_encoding(X_emb: np.ndarray, PE: np.ndarray):
 """X_emb: (B,T,D), PE: (T,D) -> (B,T,D)."""
 T_in = X_emb.shape[1]
 X_emb_pe = X_emb + PE[:T_in,:]
 return X_emb_pe


Q7. Scaled dot-product attention with masking

Implement scaled dot-product attention:
Attention(Q,K,V) = softmax((Q @ K^T) / sqrt(dk) + mask) @ V
Inputs: Q,K,V are (B,H,T,Dh). Mask is boolean broadcastable to (B,H,T,T) where False means 'mask out'.
Requirements:
- Use a numerically stable softmax (subtract max).
- Convert boolean mask to large negative values before softmax.
Return: context (B,H,T,Dh) and attention weights (B,H,T,T).

In [13]:
import numpy as np
def softmax(x: np.ndarray, axis: int = -1):
 # TODO (stable softmax)
 x_max = np.max(x, axis=axis, keepdims=True)
 exp_x = np.exp(x - x_max)
 y = exp_x / np.sum(exp_x, axis=axis, keepdims=True)
 return y
def scaled_dot_product_attention(Q, K, V, mask=None):
 """Q,K,V: (B,H,T,Dh), mask: bool broadcastable to (B,H,T,T)."""
 d_k = Q.shape[-1]

    # 1. Dot product: (B,H,T,Dh) @ (B,H,Dh,T) -> (B,H,T,T)
 scores = np.matmul(Q, K.swapaxes(-1, -2))

 # 2. Scale
 scores = scores / np.sqrt(d_k)

 # 3. Masking
 if mask is not None:
    # Broadcast mask to scores shape if needed
    # Convert False to -1e9 (large negative)
    scores = np.where(mask, scores, -1e9)

 # 4. Softmax
 attn = softmax(scores, axis=-1)

 # 5. Multiply by V: (B,H,T,T) @ (B,H,T,Dh) -> (B,H,T,Dh)
 context = np.matmul(attn, V)

 return context, attn

Q8. Multi-head self-attention (MHA)

Implement multi-head self-attention for input X (B,T,D).
- Project to Q,K,V using weight matrices Wq,Wk,Wv each (D,D).
- Reshape/split into heads -> (B,H,T,Dh) where Dh=D/H.
- Apply scaled dot-product attention with a pad mask (B,T) (broadcast it appropriately).
- Concatenate heads and apply output projection Wo (D,D).
Return: out (B,T,D) and attention weights (B,H,T,T).

In [14]:
import numpy as np
def linear(x: np.ndarray, W: np.ndarray, b=None):
 y = np.matmul(x, W)
 if b is not None:
     y += b
 return y
def split_heads(x: np.ndarray, n_heads: int):
 """(B,T,D) -> (B,H,T,Dh)"""
 B, T, D = x.shape
 Dh = D // n_heads
 # Reshape to (B, T, H, Dh) then transpose to (B, H, T, Dh)
 x = x.reshape(B, T, n_heads, Dh)
 xh = x.transpose(0, 2, 1, 3)
 return xh
def combine_heads(xh: np.ndarray):
 """(B,H,T,Dh) -> (B,T,D)"""
 B, H, T, Dh = xh.shape
 D = H * Dh
 # Transpose to (B, T, H, Dh) then reshape
 xh = xh.transpose(0, 2, 1, 3)
 x = xh.reshape(B, T, D)
 return x
def mha_self_attention(X, Wq, Wk, Wv, Wo, n_heads: int, pad_mask=None):
 Q = linear(X, Wq)
 K = linear(X, Wk)
 V = linear(X, Wv)

 Q_h = split_heads(Q, n_heads)
 K_h = split_heads(K, n_heads)
 V_h = split_heads(V, n_heads)

 # pad_mask is (B,T). We need (B,1,1,T) for broadcasting against (B,H,T,T) scores.
 # The mask should apply to the key dimension (last T).
 attn_mask = None
 if pad_mask is not None:
  # Reshape to (B, 1, 1, T) so it broadcasts over Heads and Query-Position
   attn_mask = pad_mask[:, np.newaxis, np.newaxis, :]

 # 4. Scaled Dot Product Attention
 context_h, attn = scaled_dot_product_attention(Q_h, K_h, V_h, mask=attn_mask)

 # 5. Concatenate heads
 context = combine_heads(context_h)

 # 6. Output projection
 out = linear(context, Wo)
 return out, attn

Q9. LayerNorm + residual connection

Implement LayerNorm for X (B,T,D) using learnable gamma and beta of shape (D,).
Then implement residual_add_and_norm(Y, X, gamma, beta) that returns LayerNorm(X + Y).

In [15]:
import numpy as np
def layer_norm(X: np.ndarray, gamma: np.ndarray, beta: np.ndarray, eps: float = 1e-5):
 mean = np.mean(X, axis=-1, keepdims=True)
 var = np.var(X, axis=-1, keepdims=True)

 X_hat = (X - mean) / np.sqrt(var + eps)
 Y = gamma * X_hat + beta
 return Y
def residual_add_and_norm(Y: np.ndarray, X: np.ndarray, gamma: np.ndarray, beta: np.ndarray):
 Z = layer_norm(X + Y, gamma, beta)
 return Z

Q10. Position-wise FeedForward network

Implement FFN: FFN(X) = relu(X @ W1 + b1) @ W2 + b2
Shapes: X (B,T,D), W1 (D,Dff), b1 (Dff,), W2 (Dff,D), b2 (D,)
Return: (B,T,D).

In [17]:
import numpy as np
def relu(x: np.ndarray):
 y = np.maximum(0, x)
 return y
def feed_forward(X: np.ndarray, W1: np.ndarray, b1: np.ndarray, W2: np.ndarray, b2: np.ndarray):
 hidden = linear(X, W1, b1)
 hidden = relu(hidden)
 Y = linear(hidden, W2, b2)
 return Y

# Part C - Putting it together

Q11. One Transformer encoder block (forward)

Implement a single encoder block forward pass:
1) MHA = MultiHeadSelfAttention(X) with pad_mask
2) X1 = LayerNorm(X + MHA)
3) FFN = FeedForward(X1)
4) X2 = LayerNorm(X1 + FFN)
Return X2.
You may pass all parameters explicitly (weights, gamma/beta).

In [18]:
def encoder_block_forward(X, params, n_heads: int, pad_mask=None):
 """X: (B,T,D). params is a dict of numpy arrays."""
 Wq, Wk, Wv, Wo = params['Wq'], params['Wk'], params['Wv'], params['Wo']

 # 1. MHA
 mha_out, _ = mha_self_attention(X, Wq, Wk, Wv, Wo, n_heads, pad_mask)

 # 2. Add & Norm 1
 # Note: Using 'gamma1', 'beta1' for the first LN
 X1 = residual_add_and_norm(mha_out, X, params['gamma1'], params['beta1'])

 # 3. FFN
 ffn_out = feed_forward(X1, params['W1'], params['b1'], params['W2'], params['b2'])

 # 4. Add & Norm 2
 X2 = residual_add_and_norm(ffn_out, X1, params['gamma2'], params['beta2'])
 return X2

Q12. Sequence classification head + end-to-end demo

Create an end-to-end forward pass for a tiny classifier:
- Input ids -> embeddings + positional enc
- One encoder block
- Pooling: take the [BOS] position (t=0) as the sequence representation
- Linear head: logits = h0 @ Wcls + bcls with Wcls (D,2), bcls (2,)
- Softmax to probabilities
Write predict_proba that takes a batch of texts and returns probs (B,2).
Include simple sanity checks: shapes, probabilities sum to 1, and masking doesn't crash for different
lengths.


In [20]:
def predict_proba(texts, token2id, E, PE, params, Wcls, bcls, n_heads: int):
 """Return probs: (B,2)."""
 id_seqs = []
 for text in texts:
    clean = clean_text(text)
    tokens = clean.split()
    # Prepend BOS token
    tokens = ['[BOS]'] + tokens
    ids = tokens_to_ids(tokens, token2id)
    id_seqs.append(ids)

 # 2. Batching
 pad_id = token2id['[PAD]']
 X_ids, pad_mask = pad_batch(id_seqs, pad_id) # X_ids: (B,T)

 # 3. Embedding
 X_emb = embed(X_ids, E) # (B,T,D)

 # 4. Positional Encoding
 X_emb = add_positional_encoding(X_emb, PE) # (B,T,D)

 # 5. Encoder Block
 # params dictionary must contain all weights for MHA, LN, and FFN
 X_enc = encoder_block_forward(X_emb, params, n_heads, pad_mask) # (B,T,D)

 # 6. Pooling (take [BOS] position at t=0)
 # Shape becomes (B, D)
 seq_repr = X_enc[:, 0, :]

 # 7. Linear head (Logits)
 # logits = h0 @ Wcls + bcls
 logits = linear(seq_repr, Wcls, bcls) # (B, 2)

 # 8. Softmax
 probs = softmax(logits, axis=-1)

 assert probs.shape == (len(texts), 2), f"Shape mismatch: {probs.shape}"
 assert np.allclose(np.sum(probs, axis=1), 1.0), "Probs do not sum to 1"
 return probs